In [1]:
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms as transforms
import json
import os
import torch
from torchvision.datasets import VisionDataset
from torchvision.datasets.utils import check_integrity, download_and_extract_archive
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets,models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
batch_size = 8
augment= False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)
ways=5
shots=5

cuda


In [2]:
import gc
gc.collect()
torch.cuda.empty_cache()

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_dataset=datasets.ImageFolder(root="/home/siu856542507/zarif/FederatedMeta/2750")

#   transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
#         ]) weights="IMAGENET1K_V2"

In [4]:
from torch.utils.data import random_split
from torch.utils.data import Dataset

random_seed = 22
torch.manual_seed(random_seed)
# val_size = 2500
test_size = 2500
train_size = len(train_dataset)  - test_size

train_ds, test_ds  = random_split(train_dataset, [train_size, test_size])
len(train_ds), len(test_ds)

(24500, 2500)

In [5]:
train_transform = transforms.Compose([transforms.Resize(32),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
                                 ])

test_transform = transforms.Compose([transforms.Resize(32),
                                 transforms.ToTensor(),
                                transforms.Normalize([0.3444, 0.3803, 0.4078], [0.2027, 0.1369, 0.1156])
                                    ])
class ApplyTransform(Dataset):
    def __init__(self, dataset, transform=None, target_transform=None):
        self.dataset = dataset
        self.transform = transform
        self.target_transform = target_transform
        if transform is None and target_transform is None:
            print("Transform is not implemented :)")

    def __getitem__(self, idx):
        sample, target = self.dataset[idx]
        if self.transform is not None:
            sample = self.transform(sample)
        if self.target_transform is not None:
            target = self.target_transform(target)
        return sample, target

    def __len__(self):
        return len(self.dataset)

train_ds = ApplyTransform(train_ds, train_transform)
test_ds = ApplyTransform(test_ds,test_transform)


In [6]:
import torch
from torchvision import datasets
from torchvision import transforms
import matplotlib.pyplot as plt

# transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),


def non_iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle, shuffle_digits=False):
    assert(nb_nodes>0 and nb_nodes<=10)

    digits=torch.arange(10) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))
    digits2=torch.arange(3) if shuffle_digits==False else torch.randperm(10, generator=torch.Generator().manual_seed(0))

    digits = torch.cat((digits,digits2))

    print(digits)
   
    digits_split=list()
    i=0
    for n in range(nb_nodes, 0, -1):
        inc=int((10-i)/n)
        digits_split.append(digits[i:i+3])
        print
        i+=inc
        print(inc)
        print(digits_split)
    
    print(digits_split)
    # load and shuffle nb_nodes*n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=nb_nodes*n_samples_per_node,
                                        shuffle=shuffle)
    dataiter = iter(loader)
    images_train_mnist, labels_train_mnist = next(dataiter)

    data_splitted=list()
    for i in range(nb_nodes):
        idx=torch.stack([y_ == labels_train_mnist for y_ in digits_split[i]]).sum(0).bool() # get indices for the digits
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(images_train_mnist[idx], labels_train_mnist[idx]), batch_size=batch_size, shuffle=shuffle))
    print(data_splitted)
    return data_splitted

def iid_split(dataset, nb_nodes, n_samples_per_node, batch_size, shuffle):
    # load and shuffle n_samples_per_node from the dataset
    loader = torch.utils.data.DataLoader(dataset,
                                        batch_size=n_samples_per_node,
                                        shuffle=shuffle)
    dataiter = iter(loader)

    data_splitted=list()

    for _ in range(nb_nodes):
        data_splitted.append(torch.utils.data.DataLoader(torch.utils.data.TensorDataset(*(next(dataiter))), batch_size=batch_size, shuffle=shuffle))

    return data_splitted


def  get_violence(type, n_samples_train, n_samples_test, n_clients, batch_size, shuffle,train_dataset , test_dataset):

        if type=="iid":
            train=iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        elif type=="non_iid":
            train=non_iid_split(train_dataset, n_clients, n_samples_train, batch_size, shuffle)
            test=non_iid_split(test_dataset, n_clients, n_samples_test, batch_size, shuffle)
        else:
            train=[]
            test=[]

        return train, test
    
def  get_MNIST(type="iid", n_samples_train=200, n_samples_test=100, n_clients=3, batch_size=25, shuffle=True):
    dataset_loaded_train = datasets.MNIST(
            root="./data",
            train=True,
            download=True,
            transform=transforms.ToTensor()
    )
    dataset_loaded_test = datasets.MNIST(
            root="./data",
            train=False,
            download=True,
            transform=transforms.ToTensor()
    )

    if type=="iid":
        train=iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    elif type=="non_iid":
        train=non_iid_split(dataset_loaded_train, n_clients, n_samples_train, batch_size, shuffle)
        test=non_iid_split(dataset_loaded_test, n_clients, n_samples_test, batch_size, shuffle)
    else:
        train=[]
        test=[]

    return train, test
def plot_samples(data, channel:int, title=None, plot_name="", n_examples =20):

    n_rows = int(n_examples / 5)
    plt.figure(figsize=(1* n_rows, 1*n_rows))
    if title: plt.suptitle(title)
    X, y= data
    for idx in range(n_examples):
        
        ax = plt.subplot(n_rows, 5, idx + 1)

        image = 255 - X[idx, channel].view((64,64))
        ax.imshow(image, cmap='gist_gray')
        ax.axis("off")
    if plot_name!="":plt.savefig(f"plots/"+plot_name+".png")
    plt.tight_layout()

In [7]:
mnist_iid_train_dls, mnist_iid_test_dls = get_violence("iid",
    n_samples_train =500, n_samples_test=300, n_clients =3, 
    batch_size = batch_size, shuffle =True,train_dataset=train_ds,test_dataset=test_ds)

In [8]:

# mnist_iid_train_dls, mnist_iid_test_dls = get_MNIST("iid",
#     n_samples_train =500, n_samples_test=100, n_clients =10, 
#     batch_size =batch_size, shuffle =True)

In [9]:
# def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1):
#     return nn.Sequential(
#         nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
#         nn.BatchNorm2d(out_channels),
#         nn.ReLU(),
#         nn.MaxPool2d(kernel_size=2, stride=1)
#     )

# model = nn.Sequential(
#         conv_block(3, 64,3),
#         conv_block(64, 128,3),
#         conv_block(128, 256,3),
#         conv_block(256, 512,3),
#         nn.AdaptiveAvgPool2d((1,1)),
#         nn.Flatten(),
#         nn.Linear(512, 128),
#         nn.ReLU(),
#         nn.Dropout(0.3),
#         nn.Linear(128, 64),
#         nn.ReLU(),
#         nn.Dropout(0.3),
#         nn.Linear(64, ways),
#     )
        


# model=model.to(device)


In [10]:

model =models.resnet18(weights='IMAGENET1K_V1')

# for param in model.parameters():
#     param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, ways)

# # Transfer the model to device
model = model.to(device)
# Transfer the model to device


In [11]:
# class MnistResNet(nn.Module):
#   def __init__(self, in_channels=3):
#     super(MnistResNet, self).__init__()

#     # Load a pretrained resnet model from torchvision.models in Pytorch
#     self.model = models.resnet50(pretrained=True)

#     # Change the input layer to take Grayscale image, instead of RGB images. 
#     # Hence in_channels is set as 1 or 3 respectively
#     # original definition of the first layer on the ResNet class
#     # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#     self.model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
#     # Change the output layer to output 10 classes instead of 1000 classes
#     num_ftrs = self.model.fc.in_features
#     self.model.fc = nn.Linear(num_ftrs, 14)

#   def forward(self, x):
#     return self.model(x)


# my_resnet = MnistResNet()

# # input = torch.randn((16,1,244,244))
# # output = my_resnet(input)

# model = MnistResNet().to(device)


In [12]:
import learn2learn as l2l

In [13]:
training_tasks=[]
testing_tasks=[]

for i in range (len(mnist_iid_train_dls)):
                
    train_dataset = l2l.data.MetaDataset(mnist_iid_train_dls[i].dataset)
#     valid_dataset = l2l.data.MetaDataset(valid_dataset)

    train_transforms = [
        l2l.data.transforms.NWays(train_dataset, ways),
        l2l.data.transforms.KShots(train_dataset, 2*shots),
        l2l.data.transforms.LoadData(train_dataset),
        l2l.data.transforms.RemapLabels(train_dataset),
        l2l.data.transforms.ConsecutiveLabels(train_dataset),
    ]
    train_tasks = l2l.data.TaskDataset(train_dataset,
                                       task_transforms=train_transforms,
                                       num_tasks=1000)
    training_tasks.append(train_tasks)
#     valid_transforms = [
#         l2l.data.transforms.NWays(valid_dataset, ways),
#         l2l.data.transforms.KShots(valid_dataset, 2*shots),
#         l2l.data.transforms.LoadData(valid_dataset),
#         l2l.data.transforms.ConsecutiveLabels(train_dataset),
#         l2l.data.transforms.RemapLabels(valid_dataset),
#     ]
#     valid_tasks = l2l.data.TaskDataset(valid_dataset,
#                                        task_transforms=valid_transforms,
#                                        num_tasks=600)
   

In [14]:
for i in range (len(mnist_iid_test_dls)):

    test_dataset = l2l.data.MetaDataset(mnist_iid_test_dls[i].dataset)
    test_transforms = [
        l2l.data.transforms.NWays(test_dataset, ways),
        l2l.data.transforms.KShots(test_dataset, 2*shots),
        l2l.data.transforms.LoadData(test_dataset),
        l2l.data.transforms.RemapLabels(test_dataset),
        l2l.data.transforms.ConsecutiveLabels(test_dataset),
    ]
    test_tasks = l2l.data.TaskDataset(test_dataset,
                                      task_transforms=test_transforms,
                                      num_tasks=600)
    testing_tasks.append(test_tasks)


In [15]:
import gc

gc.collect()
torch.cuda.empty_cache()

import warnings
warnings.filterwarnings('ignore')

In [16]:
# change loss calculation with simple loss and accuracy function not fast adapt 
# meta model update on each iteration --> check parameters
def loss_classifier(predictions,labels):
    
    loss = nn.CrossEntropyLoss(reduction='mean')
    
    return loss(predictions ,labels)


def accuracy(predictions, targets):
    predictions = predictions.argmax(dim=1).view(targets.shape)
    acc= (predictions == targets).sum().float() / targets.size(0)
    return acc


def fast_adapt(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = torch.zeros(data.size(0)).byte()
    adaptation_indices[torch.arange(shots*ways) * 2] = 1
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[1 - adaptation_indices], labels[1 - adaptation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        train_error /= len(adaptation_data)
        learner.adapt(train_error)
    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    valid_error /= len(evaluation_data)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    
    return valid_error, valid_accuracy
def fast_adapt_metrics(batch, learner, loss, adaptation_steps, shots, ways, device):

    data, labels = batch
    data, labels = data.to(device), labels.to(device)
    # Separate data into adaptation/evalutation sets
    adaptation_indices = torch.zeros(data.size(0)).byte()
    adaptation_indices[torch.arange(shots*ways) * 2] = 1
    adaptation_data, adaptation_labels = data[adaptation_indices], labels[adaptation_indices]
    evaluation_data, evaluation_labels = data[1 - adaptation_indices], labels[1 - adaptation_indices]

    # Adapt the model
    for step in range(adaptation_steps):
        train_error = loss(learner(adaptation_data), adaptation_labels)
        # train_error /= len(adaptation_data)
        learner.adapt(train_error)
        
    # Evaluate the adapted model
    predictions = learner(evaluation_data)
    valid_error = loss(predictions, evaluation_labels)
    # valid_error /= len(evaluation_data)
    valid_accuracy = accuracy(predictions, evaluation_labels)
    # valid_accuracy = l2l.utils.accuracy(predictions, evaluation_labels)

    return valid_error,valid_accuracy
def loss_dataset(model, dataset, loss_f):
    """Compute the loss of `model` on `dataset`"""
    loss=0
    
    for idx,(features,labels) in enumerate(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        loss+=loss_f(predictions,labels)
    
    loss/=idx+1
    return loss


def accuracy_dataset(model, dataset):
    
    correct=0
    
    for features,labels in iter(dataset):
        features = features.to(device)
        labels=labels.to(device)
        predictions= model(features)
        _,predicted=predictions.max(1,keepdim=True)
        correct+=torch.sum(predicted.view(-1,1)==labels.view(-1, 1)).item()
    accuracy = 100*correct/len(dataset.dataset)
    return accuracy


def train_step(model, mu:int, optimizer, train_data, loss_f,loss,meta_batch_size,
        adaptation_steps):
    total_loss=0
    meta_train_error = 0.0
    meta_train_accuracy = 0.0
    meta_valid_error = 0.0
    meta_valid_accuracy = 0.0
    meta_test_error = 0.0
    meta_test_accuracy = 0.0
    for task in range(meta_batch_size):
        learner = model.clone()
        batch=train_data.sample()

        evaluation_error,evaluation_accuracy= fast_adapt(batch,
                                                               learner,
                                                               loss,
                                                               adaptation_steps,
                                                               shots,
                                                               ways,
                                                               device)
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        meta_train_accuracy += evaluation_accuracy.item()
        
#     for idx, (features,labels) in enumerate(train_data):
#         optimizer.zero_grad()
#         features = features.to(device)
#         predictions= model(features)
#         labels=labels.to(device)
#         loss=loss_f(predictions,labels)
#         loss+=mu/2*difference_models_norm_2(model,model_0)
#         total_loss+=loss
#         loss.backward()
#         optimizer.step()
#     print("meta accuracy: ", meta_train_accuracy/meta_batch_size)    
    return meta_train_error/meta_batch_size

def meta_metrics(model,  optimizer, train_data, loss,meta_batch_size,
        adaptation_steps):
 
    losses=[]
    accuracy=[]
    for k in range(len(train_data)):
        total_loss=0
        meta_train_error = 0.0
        meta_train_accuracy = 0.0
        meta_valid_error = 0.0
        meta_valid_accuracy = 0.0
        meta_test_error = 0.0
        meta_test_accuracy = 0.0
        for task in range(meta_batch_size):
            learner = model.clone()
            batch= train_data[k].sample()

            evaluation_error, evaluation_accuracy = fast_adapt_metrics(batch,
                                                                   learner,
                                                                   loss,
                                                                   adaptation_steps,
                                                                   shots,
                                                                   ways,
                                                                   device)
            meta_train_error += evaluation_error.item()
            meta_train_accuracy += evaluation_accuracy.item()
        losses.append(meta_train_error/meta_batch_size)
        accuracy.append(meta_train_accuracy/meta_batch_size)
        print(meta_train_accuracy/meta_batch_size)
    return losses,accuracy



def local_learning(model, mu:float, optimizer, train_data, epochs:int, loss_f,loss,meta_batch_size,
        adaptation_steps):
    
#     model_0=deepcopy(model)
    
    for e in range(epochs):
        optimizer.zero_grad()
        local_loss=train_step(model,mu,optimizer,train_data,loss_f,loss,meta_batch_size,
        adaptation_steps)

        for p in model.parameters():
            p.grad.data.mul_(1.0 / meta_batch_size)
        optimizer.step()
    return local_loss


def difference_models_norm_2(model_1, model_2):
    """Return the norm 2 difference between the two model parameters
    """
    
    tensor_1=list(model_1.parameters())
    tensor_2=list(model_2.parameters())
    
    norm=sum([torch.sum((tensor_1[i]-tensor_2[i])**2) 
        for i in range(len(tensor_1))])
    
    return norm


def set_to_zero_model_weights(model):
    """Set all the parameters of a model to 0"""

    for layer_weigths in model.parameters():
        layer_weigths.data.sub_(layer_weigths.data)

In [17]:
def average_models(model, clients_models_hist:list , weights:list):

    """Creates the new model of a given iteration with the models of the other

    clients"""
    
    new_model=deepcopy(model)
    set_to_zero_model_weights(new_model)

    for k,client_hist in enumerate(clients_models_hist):
        
        for idx, layer_weights in enumerate(new_model.parameters()):

            contribution=client_hist[idx].data*weights[k]
            layer_weights.data.add_(contribution)
            
    return new_model
    

# Optimization Process

In [18]:

def FedProx(model,training_sets:list, n_iter:int, testing_sets:list, lr=0.0005, meta_lr=0.01,  mu=0, 
    file_name="test", epochs=5,meta_batch_size=8,adaptation_steps=5,):
    """ all the clients are considered in this implementation of FedProx
    Parameters:
        - `model`: common structure used by the clients and the server
        - `training_sets`: list of the training sets. At each index is the 
            training set of client "index"
        - `n_iter`: number of iterations the server will run
        - `testing_set`: list of the testing sets. If [], then the testing
            accuracy is not computed
        - `mu`: regularization term for FedProx. mu=0 for FedAvg
        - `epochs`: number of epochs each client is running
        - `lr`: learning rate of the optimizer
        - `decay`: to change the learning rate at each iteration
 
    """
    loss_hist=[]
    acc_hist=[]
    loss_f=loss_classifier
    model= l2l.algorithms.MetaSGD(model, lr=meta_lr)
    local_optimizer = optim.Adam(model.parameters(),lr=lr)
    loss = nn.CrossEntropyLoss(reduction='mean')
    
    #Variables initialization
    K=len(training_sets) #number of clients
    n_samples=sum([len(db.dataset) for db in training_sets])
    weights=([len(db.dataset)/n_samples for db in training_sets])    
    losses,acc= meta_metrics(model,  local_optimizer, training_sets, loss,meta_batch_size,
        adaptation_steps)
    loss_hist.append(losses)
    acc_hist.append(acc)
    print(loss_hist)
 # CHANGE FOR META LEARNING
#     loss_hist=[[float(loss_dataset(model, dl, loss_f).detach()) 
#         for dl in training_sets]]
#     print(loss_hist)
    
#     acc_hist=[[accuracy_dataset(model, dl) for dl in testing_sets]]
 # CHANGE FOR META LEARNING

    server_hist=[[tens_param.detach().cpu().numpy()
        for tens_param in list(model.parameters())]]
    models_hist = []

    server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
    server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])
    print(f'====> i: 0 Loss: {server_loss} Server Test Accuracy: {server_acc}')
    server_loss_list=[]
    server_accuracy_list=[]  
    
    for i in range(n_iter):
        
        clients_params=[]
        clients_models=[]
        clients_losses=[]
        
        for k in range(K):

            local_model=deepcopy(model) #meta sgd model
#             local_optimizer=optim.SGD(local_model.parameters(), lr=0.001, momentum=0.9)
#             exp_lr_scheduler = lr_scheduler.StepLR(local_optimizer, step_size=7, gamma=0.1)
#             local_model= l2l.algorithms.MetaSGD(local_model, lr=meta_lr)
                
            local_optimizer = optim.Adam(local_model.parameters(),lr=lr) #meta sgd
            loss = nn.CrossEntropyLoss(reduction='mean')    
            local_loss=local_learning(local_model,mu,local_optimizer,
                training_sets[k],epochs,loss_f,loss,meta_batch_size,adaptation_steps)
            clients_losses.append(local_loss)
                
            #GET THE PARAMETER TENSORS OF THE MODEL
            list_params=list(local_model.parameters())
            list_params=[tens_param.detach() for tens_param in list_params]
            clients_params.append(list_params)    
            clients_models.append(deepcopy(local_model))

            print(f"{k}---local_loss--- {local_loss}" )
        
        
        model = average_models(deepcopy(model), clients_params, 
            weights=weights)
        models_hist.append(clients_models)
        
        #COMPUTE THE LOSS/ACCURACY OF THE DIFFERENT CLIENTS WITH THE NEW MODEL
        # Function TO CHANGE FOR META LEARNING
#         loss_hist+=[[float(loss_dataset(model, dl, loss_f).detach()) 
#             for dl in training_sets]]
#         acc_hist+=[[accuracy_dataset(model, dl) for dl in testing_sets]]
        losses,acc= meta_metrics(model, local_optimizer, testing_sets, loss,meta_batch_size,
        adaptation_steps)
        loss_hist.append(losses)
        acc_hist.append(acc)
        server_loss=sum([weights[i]*loss_hist[-1][i] for i in range(len(weights))])
        server_acc=sum([weights[i]*acc_hist[-1][i] for i in range(len(weights))])

        print(f'====> i: {i+1} Loss: {server_loss} Server Test Accuracy: {server_acc}')
        server_accuracy_list.append(server_acc)
        server_loss_list.append(server_loss)        

        server_hist.append([tens_param.detach().cpu().numpy() 
            for tens_param in list(model.parameters())])
    return model, loss_hist, acc_hist,server_accuracy_list,server_loss_list


In [19]:
import time
start_time = time.time()

n_iter=40
# mnist_iid_train_dls, mnist_iid_test_dls
model_f, loss_hist_FA_iid, acc_hist_FA_iid,server_accuracy_list,server_loss_list = FedProx( model, training_tasks, 
                                                                                           n_iter, testing_tasks,
   lr=0.0005, meta_lr=0.01, epochs =50,meta_batch_size=batch_size,
        adaptation_steps=5,)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time: ", elapsed_time) 
def plot_acc_loss(title:str, loss_hist:list, acc_hist:list):
    plt.figure()
    
    plt.suptitle(title)

    plt.subplot(1,2,1)
    lines=plt.plot(loss_hist)
    plt.title("Loss")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])

    plt.subplot(1,2,2)
    lines=plt.plot(acc_hist )
    plt.title("Accuracy")
    plt.legend(lines,["C1", "C2", "C3","C4", "C5", "C6","C7", "C8", "C9","C10"])
    


import pickle


with open('acc-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(acc_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('loss-fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(loss_hist_FA_iid, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('server_acc_hist_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_accuracy_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('server_loss_list_fedavo-lisa.pickle', 'wb') as handle:
    pickle.dump(server_loss_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
plot_acc_loss("FedAvg MNIST-AVOA-IID-Epoch-5-Client-10", loss_hist_FA_iid, acc_hist_FA_iid)


0.4299999885261059
0.5249999910593033
0.43499998562037945
[[1.431211456656456, 1.2827838063240051, 1.4114731550216675]]
====> i: 0 Loss: 1.3751561393340428 Server Test Accuracy: 0.4633333217352628
0---local_loss--- 0.054757604375481606
1---local_loss--- 0.051435474306344986
2---local_loss--- 0.04764404520392418
0.5799999870359898
0.5549999848008156
0.5049999840557575
====> i: 1 Loss: 1.3921604851881662 Server Test Accuracy: 0.5466666519641876
0---local_loss--- 0.020828805398195982
1---local_loss--- 0.017321402148809284
2---local_loss--- 0.0225201795110479
0.6549999825656414
0.6599999815225601
0.7199999764561653
====> i: 2 Loss: 1.5757484274605909 Server Test Accuracy: 0.6783333135147889
0---local_loss--- 0.004705465558799915
1---local_loss--- 0.005567531305132434
2---local_loss--- 0.003030992593267001
0.8499999642372131
0.8149999678134918
0.8149999864399433
====> i: 3 Loss: 0.83959392295219 Server Test Accuracy: 0.8266666394968827
0---local_loss--- 0.0020286806520743994
1---local_loss-

In [ ]:
n_samples=sum([len(db.dataset) for db in training_tasks])
weights=([len(db.dataset)/n_samples for db in training_tasks])    

In [ ]:
print("\x1b[31m\"red\"\x1b[0m")